In [1]:
import networkx as nx
import numpy as np
from network import ConstantBirthNode, DynamicBirthNode, Network
from reactions import Reactions, CmeParameters, OdeParameters
from simulate import simulate_gillespie, simulate_ode

In [2]:
node0 = ConstantBirthNode(60, 0.2, 0.2)
node1 = DynamicBirthNode(60, 0.2, 0.2, 60, 0.1, 0.5)
node2 = DynamicBirthNode(60, 0.1, 0.1, 60, 0.1, 0.5)

network = Network()
network.add_node(node0)
network.add_node(node1)
network.add_node(node2)
network.allocate_ids()

network.add_transport(node0, node1, rate = 0.05)
network.add_transport(node1, node0, rate = 0.05)
network.add_transport(node1, node2, rate = 0.03)
network.add_transport(node2, node1, rate = 0.03)


for node in network.nodes: print(node)
for s, d, data in network.edges(data=True):
    print(s, d, data['rate'])

reactions = Reactions(network)

print("\nReactions:")
for reaction in reactions.reactions:
    print(reaction)

ConstantBirthNode(id:0)
DynamicBirthNode(id:1)
DynamicBirthNode(id:2)
ConstantBirthNode(id:0) DynamicBirthNode(id:1) 0.05
DynamicBirthNode(id:1) ConstantBirthNode(id:0) 0.05
DynamicBirthNode(id:1) DynamicBirthNode(id:2) 0.03
DynamicBirthNode(id:2) DynamicBirthNode(id:1) 0.03

Reactions:
ConstantBirthReaction(variable:n0_t0, state_i:0, rate:0.2)
ConstantBirthReaction(variable:n0_t1, state_i:1, rate:0.2)
DynamicBirthReaction(variable:n1_t0, state_i:2, rate:dynamic)
DynamicBirthReaction(variable:n1_t1, state_i:3, rate:dynamic)
DynamicBirthReaction(variable:n2_t0, state_i:4, rate:dynamic)
DynamicBirthReaction(variable:n2_t1, state_i:5, rate:dynamic)
DeathReaction(variable:n0_t0, state_i:0, rate:0.2)
DeathReaction(variable:n0_t1, state_i:1, rate:0.2)
DeathReaction(variable:n1_t0, state_i:2, rate:0.2)
DeathReaction(variable:n1_t1, state_i:3, rate:0.2)
DeathReaction(variable:n2_t0, state_i:4, rate:0.1)
DeathReaction(variable:n2_t1, state_i:5, rate:0.1)
TransportReaction(variable:n0_t0, state_

In [3]:
MAX_T = 100
TIME_POINTS = np.linspace(0, MAX_T, 101)
REPLICATES = 1

start_statevec = np.array([40, 40, 40, 40, 40, 40])

In [4]:
cme_param = CmeParameters(reactions)
print(cme_param)

gill_results = simulate_gillespie(cme_param, TIME_POINTS, start_statevec, REPLICATES)


Reaction Matrix:
[[ 1  0  0  0  0  0]
 [ 0  1  0  0  0  0]
 [ 0  0  1  0  0  0]
 [ 0  0  0  1  0  0]
 [ 0  0  0  0  1  0]
 [ 0  0  0  0  0  1]
 [-1  0  0  0  0  0]
 [ 0 -1  0  0  0  0]
 [ 0  0 -1  0  0  0]
 [ 0  0  0 -1  0  0]
 [ 0  0  0  0 -1  0]
 [ 0  0  0  0  0 -1]
 [-1  0  1  0  0  0]
 [ 0 -1  0  1  0  0]
 [ 1  0 -1  0  0  0]
 [ 0  1  0 -1  0  0]
 [ 0  0 -1  0  1  0]
 [ 0  0  0 -1  0  1]
 [ 0  0  1  0 -1  0]
 [ 0  0  0  1  0 -1]]
Reaction Rates:
[ 0.2   0.2  -1.   -1.   -1.   -1.    0.2   0.2   0.2   0.2   0.1   0.1
  0.05  0.05  0.05  0.05  0.03  0.03  0.03  0.03]
Statevec i:
[0 1 2 3 4 5 0 1 2 3 4 5 0 1 2 3 2 3 4 5]
Dynamic Birth Parameters:
[2 4]
[0.  0.  0.2 0.2 0.1 0.1 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0. ]
[ 0.  0. 60. 60. 60. 60.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]
[ 0.  0. 60. 60. 60. 60.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]
[0.  0.  0.5 0.5 0.5 0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0. ]

Simulat

100%|██████████| 1/1 [00:00<00:00, 10.20it/s]


In [4]:
ode_param = OdeParameters(reactions)
print(ode_param)

ode_results = simulate_ode(ode_param, TIME_POINTS, start_statevec)

array([[[40, 37, 39, 48, 44, 46, 51, 52, 47, 53, 57, 58, 54, 58, 52, 53,
         59, 68, 74, 70, 64, 61, 71, 75, 79, 79, 80, 77, 79, 80, 76, 70,
         71, 69, 76, 74, 78, 76, 79, 74, 71, 70, 65, 66, 57, 54, 43, 42,
         38, 37, 38, 35, 33, 31, 39, 35, 33, 35, 33, 30, 29, 27, 29, 36,
         38, 40, 40, 41, 43, 42, 39, 39, 38, 41, 34, 33, 36, 38, 32, 35,
         42, 47, 43, 47, 49, 48, 54, 55, 55, 59, 64, 63, 67, 63, 63, 58,
         57, 53, 50, 44, 37],
        [40, 39, 37, 31, 30, 33, 33, 32, 34, 29, 33, 34, 40, 41, 51, 49,
         45, 48, 49, 51, 60, 57, 56, 50, 56, 54, 40, 44, 51, 49, 43, 36,
         37, 36, 39, 37, 27, 32, 29, 23, 24, 22, 20, 23, 23, 26, 22, 27,
         26, 28, 25, 24, 23, 22, 20, 21, 22, 23, 14, 11, 12, 13, 17, 13,
         15, 16, 16, 15, 16, 17, 17, 12, 13, 14,  8,  6,  7,  5,  4,  5,
          6,  4,  4,  3,  6,  9,  7,  6,  3,  2,  0,  0,  0,  0,  0,  2,
          2,  2,  1,  2,  1],
        [40, 43, 42, 36, 38, 42, 42, 43, 45, 45, 45, 45, 45, 47,

global ODE_model
def ODE_model(t, z):
# Variables (<node.id>_<etype>):
	n0_t0, n0_t1, n1_t0, n1_t1, n2_t0, n2_t1 = z
# Terms for each variable:
	return [
# Δn0_t0/Δt	<birth>    <death>    <outflow>      <inflow>
		+n0_t0*0.2 -n0_t0*0.2 -n0_t0*(+0.05) +(+n1_t0*0.05) ,
# Δn0_t1/Δt	<birth>    <death>    <outflow>      <inflow>
		+n0_t1*0.2 -n0_t1*0.2 -n0_t1*(+0.05) +(+n1_t1*0.05) ,
# Δn1_t0/Δt	<birth>                                 <death>    <outflow>           <inflow>
		+n1_t0*(0.2+0.1*(60-n1_t0-(0.5*n1_t1))) -n1_t0*0.2 -n1_t0*(+0.05+0.03) +(+n0_t0*0.05+n2_t0*0.03) ,
# Δn1_t1/Δt	<birth>                                 <death>    <outflow>           <inflow>
		+n1_t1*(0.2+0.1*(60-n1_t0-(0.5*n1_t1))) -n1_t1*0.2 -n1_t1*(+0.05+0.03) +(+n0_t1*0.05+n2_t1*0.03) ,
# Δn2_t0/Δt	<birth>                                 <death>    <outflow>      <inflow>
		+n2_t0*(0.1+0.1*(60-n2_t0-(0.5*n2_t1))) -n2_t0*0.1 -n2_t0*(+0.03) +(+n1_t0*0.03) ,
# Δn2_t1/Δt	<birth>                                 <death> 

In [6]:
'''
This is a very very very extremely hacky function that works by collecting the parameters
of the differential equations from the parameters encoded in the network, uses this to make
a text python program, and then evaluates this python program and returns the output. 
'''
def ode_from_network(G, prnt = False):
    # collect data needed to generate the code
    nodenames, compartments = names_from_network(G)
    df_nodes, df_edges = dataframes_from_network(G, prnt=False) 

    # names of the variables (node names + wt and mt)
    vars = []
    for node in compartments:
        vars.append(f'{node}_wt')
        vars.append(f'{node}_mt')

    # for each variable, get the expression for how much it changes in a given time t
    diff_exp = ""    
    for var in vars: 
        node_type = var[-2:]
        node_name = var[:-3]
        var_wt_name = f'{node_name}_wt'
        var_mt_name = f'{node_name}_mt'
        
        # get corresponding node row in row dataframe
        noderow = df_nodes.loc[df_nodes['node'] == node_name]
        
        # collect numeric values of specific parameters
        cb = float(noderow['c_b'])
        birthrate = float(noderow['birth_rate'])
        nss = int(noderow['nss'])
        delta = float(noderow['delta'])
        deathrate = float(noderow['death_rate'])

        # generate birth rate
        if   int(noderow["birth_type"]) == 0:
            birth_term = '0'
        elif int(noderow["birth_type"]) == 1:
            birth_term = f"({birthrate})"
        elif int(noderow["birth_type"]) == 2:
            birth_term = f"({birthrate} + {cb}*({nss}-{var_wt_name}-({delta}*{var_mt_name})))"
            
        # generate death rate
        death_term = f"({deathrate})"
        
        # generate terms for outflow
        edges_outrow = df_edges[df_edges['source'] == node_name]
        
        out_term = f"({round(edges_outrow['rate'].sum(), 8)})"

        # generate terms for inflow
        edges_inrow = df_edges[df_edges['target'] == node_name]
        in_term = ''
        for index, row in edges_inrow.iterrows():
            in_term += f"+({str(row['source'])}_{node_type}*{float(row['rate'])})"
        
        diff_exp += f"\t\t# Δ{var}/Δt\n\t\t({var}*({birth_term}-{death_term}-{out_term})){in_term},\n"

    # set up function
    fulltext = "global ODE_model\ndef ODE_model(t, z):\n"

    # set up tuple of variables
    vars_for_code = str((str(vars)[1:-1]).replace("'",""))
    fulltext += f'\t# variables (node name + wt/mt)\n\t{vars_for_code} = z'
    
    # set up how each variable changes
    fulltext += f"\n\treturn [\n{diff_exp}\t\t]"
    
    if prnt == True:
        print(">> Code for ODE model:\n")
        print(fulltext)

    # execute the code generated by the process
    global ODE_model
    ODE_model = None
    exec(fulltext)

    return ODE_model